<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ecommerce_transaction.csv'
data = pd.read_csv(url)

In [61]:
# [1] datetime

import datetime

data1 = data

# create datetime type review_creation_date column
data1 = data1.assign(dt_review=data1.apply(lambda x:
                                 datetime.datetime.strptime(x['review_creation_date'],
                                                            '%Y-%m-%d %H:%M:%S'),
                                 axis=1))

def checkError(x):
    if x.strftime('%Y-%m-%d') == '2000-01-01':
        return '2000'
    elif x.strftime('%Y-%m-%d') < '2011-12-31':
        return '7Y'
    else:
        return 'normal'
    
# create flag for wrong review_creation_date
data1 = data1.assign(check=data1.apply(lambda x: checkError(x['dt_review']), axis=1))

# find the number of wrong records
wrong = data1[(data1['check'] == '2000') | (data1['check'] == '7Y')]['check'].count()

print(f'Answer [1] : no of wrong review date = {wrong}')

# remove type 1 wrong data
data1 = data1[data1['check'] != '2000']

def revert(x, y):
    if y == '7Y':
        year = str(int(x.strftime('%Y')) + 7)
    else:
        year = x.strftime('%Y')
    month = x.strftime('%m')
    day = x.strftime('%d')
    hour = x.strftime('%H')
    minute = x.strftime('%M')
    second = x.strftime('%S')
    dt = year + '-' + month + '-' + day + ' ' + hour + ':' + minute + ':' + second
    #return datetime.datetime.strptime(dt, '%Y-%m-%d %H:%m:%S')
    return dt

# correct type 2 wrong data
data1['review_creation_date'] = data1.apply(lambda x: revert(x['dt_review'],
                                                             x['check']),
                                            axis=1)

# update dt_review with corrected review_creation_date
data1['dt_review'] = data1.apply(lambda x:
                                 datetime.datetime.strptime(x['review_creation_date'],
                                                            '%Y-%m-%d %H:%M:%S'),
                                 axis=1)
data1 = data1.drop(axis=1, columns='check')

Answer [1] : no of wrong review date = 95


In [62]:
# [2]

data2 = data1

# convert str timestamp to timestamp type
data2 = data2.assign(dt_order=data2.apply(lambda x:
                                          datetime.datetime.strptime(x['order_timestamp'],
                                                                     '%Y-%m-%d %H:%M:%S'),
                                          axis=1))
data2 = data2.assign(dt_approve=data2.apply(lambda x:
                                            datetime.datetime.strptime(x['order_approved_time'],
                                                                       '%Y-%m-%d %H:%M:%S'),
                                            axis=1))

# calculate approval lead time
def leadtime(x, y):
    gap = y - x
    lt_seconds = gap.days * 24 * 60 * 60 + gap.seconds
    return lt_seconds

data2 = data2.assign(leadtime=data2.apply(lambda x: leadtime(x['dt_order'],
                                                             x['dt_approve']),
                                          axis=1))
data2 = data2.assign(method=data2.apply(lambda x: '계좌이체' if x['leadtime'] >= 3600
                                        else '신용카드', axis=1))

records_by_method = data2.groupby('method').agg({'id': 'count'})
bank = records_by_method['id']['계좌이체']
card = records_by_method['id']['신용카드']

print(f'Answer [2] : Ratio = {bank / (bank + card) * 100:.1f}')

Answer [2] : Ratio = 36.1


In [63]:
# [3]

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

data3 = data2

# calculate product price
data3 = data3.assign(price=data3.apply(lambda x: x['order_price'] / x['order_qty'],
                                       axis=1))

def calcRange(x):
    if x < 50:
        return '0_50'
    elif x < 100:
        return '50_100'
    elif x < 500:
        return '100_500'
    elif x < 5000:
        return '500_5000'
    else:
        return 'ERROR'

# calculate price range
data3 = data3.assign(p_range=data3.apply(lambda x: calcRange(x['price']), axis=1))
# calculate order_price range
data3 = data3.assign(o_range=data3.apply(lambda x: calcRange(x['order_price']), axis=1))

# perform ANOVA TEST
result_amount = ols(formula='review_score ~ C(o_range)', data=data3).fit()
result_price = ols(formula='review_score ~ C(p_range)', data=data3).fit()

table_amount = anova_lm(result_amount)
table_price = anova_lm(result_price)

f_amount = table_amount['F'][0]
f_price = table_amount['F'][0]

print(f'Answer [3] : Max F-value = {max(f_amount, f_price):.2f}')


Answer [3] : Max F-value = 0.66


In [68]:
# [4]

from sklearn.linear_model import LinearRegression

data4 = data3[data3['order_status'] == 'delivered']

# convert str timestamp to timestamp type
data4 = data4.assign(dt_delivered=data4.apply(lambda x:
                                  datetime.datetime.strptime(x['order_delivered_customer_date'],
                                                            '%Y-%m-%d %H:%M:%S'),
                                          axis=1))
# calculate delivery lead time
data4 = data4.assign(del_lt=data4.apply(lambda x:
                                        leadtime(x['dt_approve'],
                                                 x['dt_delivered']) / 60 / 60 / 24,
                                        axis=1))

# calculate weekdays vs. weekends
def dayOfWeek(x):
    wd = int(x.strftime('%w'))
    if (wd == 0) | (wd == 6):
        return 'weekends'
    else:
        return 'weekdays'

data4 = data4.assign(wd=data4.apply(lambda x: dayOfWeek(x['dt_approve']),
                                    axis=1))

# Train
X_wd = data4[data4['wd'] == 'weekdays'][['order_freight']]
X_we = data4[data4['wd'] == 'weekends'][['order_freight']]
y_wd = data4[data4['wd'] == 'weekdays']['del_lt']
y_we = data4[data4['wd'] == 'weekends']['del_lt']

lr_wd = LinearRegression()
lr_we = LinearRegression()

model_wd = lr_wd.fit(X_wd, y_wd)
model_we = lr_we.fit(X_we, y_we)

print(f'Answer [4] : Coef Diff = {abs(model_wd.coef_[0] - model_we.coef_[0]):.4f}')

Answer [4] : Coef Diff = 0.0097
